In [1]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('operations').getOrCreate()


df = spark.read.csv('high_diamond_ranked_10min.csv', inferSchema=True, header=True)

In [2]:
print(df.columns)
print(len(df.columns),df.count())

['blueWardsPlaced', 'blueWardsDestroyed', 'blueFirstBlood', 'blueKills', 'blueDeaths', 'blueAssists', 'blueEliteMonsters', 'blueDragons', 'blueHeralds', 'blueTowersDestroyed', 'blueTotalGold', 'blueAvgLevel', 'blueTotalExperience', 'blueTotalMinionsKilled', 'blueTotalJungleMinionsKilled', 'blueGoldDiff', 'blueExperienceDiff', 'blueCSPerMin', 'blueGoldPerMin', 'redWardsPlaced', 'redWardsDestroyed', 'redFirstBlood', 'redKills', 'redDeaths', 'redAssists', 'redEliteMonsters', 'redDragons', 'redHeralds', 'redTowersDestroyed', 'redTotalGold', 'redAvgLevel', 'redTotalExperience', 'redTotalMinionsKilled', 'redTotalJungleMinionsKilled', 'redGoldDiff', 'redExperienceDiff', 'redCSPerMin', 'redGoldPerMin', 'blueWins']
39 9879


In [3]:
import matplotlib.pyplot as plt

v1=df.rdd.map(lambda x:x[3]).collect()
plt.hist(v1, color='blue')
plt.show()

v2=df.rdd.map(lambda x:x[38]).collect()
plt.hist(v2, color='blue')
plt.show()

v3=df.rdd.map(lambda x:x[2]).collect()
plt.hist(v3, color='blue')
plt.show()


<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [4]:
dropatt=["redFirstBlood","redGoldDiff"]
df_drop=df.drop(*dropatt)
print(len(df_drop.columns),df_drop.count())


37 9879


In [5]:
df_filter1=df_drop.filter((df_drop.blueKills <= 15)&(df_drop.blueGoldDiff<3500)&(df_drop.blueGoldDiff>-3500))
print(len(df_filter1.columns),df_filter1.count())




37 8392


In [6]:
import pyspark.sql.functions as pyfunc
df_filter2=df_filter1.withColumn("bluegoldhigh>18000",pyfunc.when(df_filter1.blueTotalGold< 18000,0).otherwise(1))
df_filter3=df_filter2.withColumn("bluelevelhigh>7",pyfunc.when(df_filter2.blueAvgLevel < 7,0).otherwise(1))
df_filter4=df_filter3.withColumn("blueexperiencehigh>18000",pyfunc.when(df_filter3.blueTotalExperience<18000,0).otherwise(1))
df_filter5=df_filter4.withColumn("bluegolddiffhigh>2000",pyfunc.when(df_filter4.blueGoldDiff <2000,0).otherwise(1))

df_filter6=df_filter5.withColumn("redgoldhigh>18000",pyfunc.when(df_filter5.redTotalGold< 18000,0).otherwise(1))
df_filter7=df_filter6.withColumn("redlevelhigh>7",pyfunc.when(df_filter6.redAvgLevel < 7,0).otherwise(1))
df_filter8=df_filter7.withColumn("redexperiencehigh>18000",pyfunc.when(df_filter7.redTotalExperience<18000,0).otherwise(1))

df_filter8.select(*["bluegoldhigh>18000","bluelevelhigh>7","blueexperiencehigh>18000","bluegolddiffhigh>2000","redgoldhigh>18000","redlevelhigh>7","redexperiencehigh>18000"]).describe().show()


+-------+-------------------+-------------------+------------------------+---------------------+-------------------+------------------+-----------------------+
|summary| bluegoldhigh>18000|    bluelevelhigh>7|blueexperiencehigh>18000|bluegolddiffhigh>2000|  redgoldhigh>18000|    redlevelhigh>7|redexperiencehigh>18000|
+-------+-------------------+-------------------+------------------------+---------------------+-------------------+------------------+-----------------------+
|  count|               8392|               8392|                    8392|                 8392|               8392|              8392|                   8392|
|   mean|0.10927073403241182| 0.5425405147759771|      0.4849857006673022|  0.14954718779790277|0.10581506196377502|0.5580314585319351|     0.4915395614871306|
| stddev|0.31199717967540275|0.49821670260872386|      0.4998042994779815|   0.3566482629594735| 0.3076191001588246|0.4966505233125514|    0.49995820452957657|
|    min|                  0|           

In [7]:
[df_filter8.corr("blueWins",x) for x in df_filter8.columns]

[-0.003021990701087143,
 0.031661367603869145,
 0.13848591013372563,
 0.232377735253931,
 -0.23590672393762963,
 0.1832807934930166,
 0.17375317597153447,
 0.17655949398197546,
 0.05871885268587442,
 0.04282587180378213,
 0.3050986899805216,
 0.25372859359725003,
 0.2906941350598277,
 0.136415036564105,
 0.10100642634560494,
 0.42926504004593397,
 0.3991095209715218,
 0.13641503656410484,
 0.3050986899805199,
 -0.01873856807684418,
 -0.041004850747467486,
 -0.23590672393762963,
 0.232377735253931,
 -0.18019440728374483,
 -0.18047575775953742,
 -0.1757741642635022,
 -0.06885522778602239,
 -0.028482035565778687,
 -0.30016378632731855,
 -0.2533933521110137,
 -0.2869026565973431,
 -0.1227211626526542,
 -0.08006705715133035,
 -0.3991095209715218,
 -0.12272116265265379,
 -0.30016378632731827,
 1.0,
 0.16105260073738414,
 0.20767759428593607,
 0.2376930263327145,
 0.2793015291197216,
 -0.15393484791169448,
 -0.21490614010690864,
 -0.23360136926551292]

In [8]:
df_filter8.columns

['blueWardsPlaced',
 'blueWardsDestroyed',
 'blueFirstBlood',
 'blueKills',
 'blueDeaths',
 'blueAssists',
 'blueEliteMonsters',
 'blueDragons',
 'blueHeralds',
 'blueTowersDestroyed',
 'blueTotalGold',
 'blueAvgLevel',
 'blueTotalExperience',
 'blueTotalMinionsKilled',
 'blueTotalJungleMinionsKilled',
 'blueGoldDiff',
 'blueExperienceDiff',
 'blueCSPerMin',
 'blueGoldPerMin',
 'redWardsPlaced',
 'redWardsDestroyed',
 'redKills',
 'redDeaths',
 'redAssists',
 'redEliteMonsters',
 'redDragons',
 'redHeralds',
 'redTowersDestroyed',
 'redTotalGold',
 'redAvgLevel',
 'redTotalExperience',
 'redTotalMinionsKilled',
 'redTotalJungleMinionsKilled',
 'redExperienceDiff',
 'redCSPerMin',
 'redGoldPerMin',
 'blueWins',
 'bluegoldhigh>18000',
 'bluelevelhigh>7',
 'blueexperiencehigh>18000',
 'bluegolddiffhigh>2000',
 'redgoldhigh>18000',
 'redlevelhigh>7',
 'redexperiencehigh>18000']

In [9]:
dropatt2=["redexperiencehigh>18000","redlevelhigh>7","redgoldhigh>18000","redGoldPerMin","redCSPerMin","redExperienceDiff","redTotalJungleMinionsKilled","redTotalMinionsKilled","redTotalExperience","redAvgLevel","redTotalGold","redTowersDestroyed","redHeralds","redDragons","redEliteMonsters","redAssists","redKills","redWardsDestroyed","redWardsPlaced","blueWardsPlaced","blueDeaths"]
df_drop2=df_filter8.drop(*dropatt2)
print(len(df_drop2.columns),df_drop2.count())
[df_drop2.corr("blueWins",x) for x in df_drop2.columns]


23 8392


[0.031661367603869145,
 0.13848591013372563,
 0.232377735253931,
 0.1832807934930166,
 0.17375317597153447,
 0.17655949398197546,
 0.05871885268587442,
 0.04282587180378213,
 0.3050986899805216,
 0.25372859359725003,
 0.2906941350598277,
 0.136415036564105,
 0.10100642634560494,
 0.42926504004593397,
 0.3991095209715218,
 0.13641503656410484,
 0.3050986899805199,
 0.232377735253931,
 1.0,
 0.16105260073738414,
 0.20767759428593607,
 0.2376930263327145,
 0.2793015291197216]

In [10]:
df_drop2.columns

['blueWardsDestroyed',
 'blueFirstBlood',
 'blueKills',
 'blueAssists',
 'blueEliteMonsters',
 'blueDragons',
 'blueHeralds',
 'blueTowersDestroyed',
 'blueTotalGold',
 'blueAvgLevel',
 'blueTotalExperience',
 'blueTotalMinionsKilled',
 'blueTotalJungleMinionsKilled',
 'blueGoldDiff',
 'blueExperienceDiff',
 'blueCSPerMin',
 'blueGoldPerMin',
 'redDeaths',
 'blueWins',
 'bluegoldhigh>18000',
 'bluelevelhigh>7',
 'blueexperiencehigh>18000',
 'bluegolddiffhigh>2000']

In [11]:
df_drop2.filter((df_drop2.blueWins == 1)).count()

4193

In [12]:
df_drop2.filter((df_drop2.blueWins == 0)).count()

4199

In [13]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler


In [14]:
df_drop2.columns

['blueWardsDestroyed',
 'blueFirstBlood',
 'blueKills',
 'blueAssists',
 'blueEliteMonsters',
 'blueDragons',
 'blueHeralds',
 'blueTowersDestroyed',
 'blueTotalGold',
 'blueAvgLevel',
 'blueTotalExperience',
 'blueTotalMinionsKilled',
 'blueTotalJungleMinionsKilled',
 'blueGoldDiff',
 'blueExperienceDiff',
 'blueCSPerMin',
 'blueGoldPerMin',
 'redDeaths',
 'blueWins',
 'bluegoldhigh>18000',
 'bluelevelhigh>7',
 'blueexperiencehigh>18000',
 'bluegolddiffhigh>2000']

In [15]:
assembler = VectorAssembler(
  inputCols=['blueWardsDestroyed',
             'blueFirstBlood',
             'blueKills',
             'blueAssists',
             'blueEliteMonsters',
             'blueDragons',
             'blueHeralds',
             'blueTowersDestroyed',
             'blueTotalGold',
             'blueAvgLevel',
             'blueTotalExperience',
             'blueTotalMinionsKilled',
             'blueTotalJungleMinionsKilled',
             'blueGoldDiff',
             'blueExperienceDiff',
             'blueCSPerMin',
             'blueGoldPerMin',
             'redDeaths',
             'bluegoldhigh>18000',
             'bluelevelhigh>7',
             'blueexperiencehigh>18000',
             'bluegolddiffhigh>2000'],
              outputCol="features")



In [16]:
output = assembler.transform(df_drop2)

In [17]:
fitmodel=output.select("features","blueWins")

In [18]:
fitmodel.show()

+--------------------+--------+
|            features|blueWins|
+--------------------+--------+
|[2.0,1.0,9.0,11.0...|       0|
|(22,[0,2,3,8,9,10...|       0|
|[0.0,0.0,7.0,4.0,...|       0|
|[1.0,0.0,4.0,5.0,...|       0|
|[4.0,0.0,6.0,6.0,...|       0|
|[0.0,0.0,5.0,6.0,...|       1|
|[3.0,1.0,7.0,7.0,...|       1|
|(22,[0,2,3,8,9,10...|       0|
|[3.0,0.0,7.0,8.0,...|       0|
|[1.0,1.0,4.0,5.0,...|       1|
|[3.0,1.0,4.0,6.0,...|       0|
|[2.0,1.0,11.0,7.0...|       0|
|[1.0,1.0,7.0,11.0...|       1|
|[3.0,0.0,4.0,1.0,...|       0|
|[3.0,1.0,4.0,4.0,...|       1|
|(22,[0,2,3,8,9,10...|       0|
|[3.0,0.0,3.0,3.0,...|       0|
|[4.0,1.0,5.0,11.0...|       1|
|[3.0,0.0,5.0,5.0,...|       0|
|[3.0,1.0,11.0,15....|       1|
+--------------------+--------+
only showing top 20 rows



In [19]:
train_data,test_data = fitmodel.randomSplit([0.9,0.1])

In [20]:
from pyspark.ml.classification import DecisionTreeClassifier,GBTClassifier,RandomForestClassifier
from pyspark.ml import Pipeline

In [21]:
dtc = DecisionTreeClassifier(labelCol='blueWins',featuresCol='features')
rfc = RandomForestClassifier(labelCol='blueWins',featuresCol='features')
gbt = GBTClassifier(labelCol='blueWins',featuresCol='features')

In [22]:
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

In [23]:
dtc_predictions = dtc_model.transform(test_data)
rfc_predictions = rfc_model.transform(test_data)
gbt_predictions = gbt_model.transform(test_data)

In [24]:
# Let's start off with binary classification.
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Note that the label column isn't named label, it's named PrivateIndex in this case.
my_binary_eval = BinaryClassificationEvaluator(labelCol = 'blueWins')

# This is the area under the curve. This indicates that the data is highly seperable.
print("DTC")
print(my_binary_eval.evaluate(dtc_predictions))

# RFC improves accuracy but also model complexity. RFC outperforms DTC in nearly every situation.
print("RFC")
print(my_binary_eval.evaluate(rfc_predictions))

# We can't repeat these exact steps for GBT. If you print the schema of all three, you may be able to notice why.
# Instead, let's redefine the object:
my_binary_gbt_eval = BinaryClassificationEvaluator(labelCol='blueWins', rawPredictionCol='prediction')
print("GBT")
print(my_binary_gbt_eval.evaluate(gbt_predictions))

DTC
0.6626321384279218
RFC
0.7606980452371095
GBT
0.6779542904387882


In [25]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
acc_evaluator = MulticlassClassificationEvaluator(labelCol="blueWins", predictionCol="prediction", metricName="accuracy")

In [26]:
dtc_acc = acc_evaluator.evaluate(dtc_predictions)
rfc_acc = acc_evaluator.evaluate(rfc_predictions)
gbt_acc = acc_evaluator.evaluate(gbt_predictions)

print("Here are the results!")
print('-'*40)
print('A single decision tree has an accuracy of: {0:2.2f}%'.format(dtc_acc*100))
print('-'*40)
print('A random forest ensemble has an accuracy of: {0:2.2f}%'.format(rfc_acc*100))
print('-'*40)
print('An ensemble using GBT has an accuracy of: {0:2.2f}%'.format(gbt_acc*100))

Here are the results!
----------------------------------------
A single decision tree has an accuracy of: 67.56%
----------------------------------------
A random forest ensemble has an accuracy of: 68.29%
----------------------------------------
An ensemble using GBT has an accuracy of: 67.80%


In [28]:
TP = dtc_predictions.filter(dtc_predictions['prediction'] == 1).filter(dtc_predictions['blueWins'] == 1).count()
FN = dtc_predictions.filter(dtc_predictions['prediction'] == 0).filter(dtc_predictions['blueWins'] == 1).count()
TN = dtc_predictions.filter(dtc_predictions['prediction'] == 0).filter(dtc_predictions['blueWins'] == 0).count()
FP = dtc_predictions.filter(dtc_predictions['prediction'] == 1).filter(dtc_predictions['blueWins'] == 0).count()

In [31]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
conf_mat = sc.parallelize([
    {"Positive": TP, "Negative": FP,"Matrix":"Positive"},
    {"Positive": FN, "Negative": TN,"Matrix":"Negative"}
]).toDF()

/home/ubuntu/spark-2.1.1-bin-hadoop2.7/python/pyspark/sql/session.py:340: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "


In [32]:
conf_mat.show()

+--------+--------+--------+
|  Matrix|Negative|Positive|
+--------+--------+--------+
|Positive|     145|     291|
|Negative|     265|     122|
+--------+--------+--------+

